XGBoost model
=====

here we try to implement the xgboost model

In [1]:
import pandas as pd
import numpy as np
%cd ../data
data_original = pd.read_csv('data_cleaned.csv',  delimiter=',', header=0)

data_original.head()

/home/sander/Documents/Projects/ids2019-m16-traffic-violations/data


,vehicle_type,vehicle_country,vehicle_brand,vehicle_year,age_group,infraction,x,y,weekday,road_type,infraction_seriousness,gender
0,car,EST,MITSUBISHI,2000,18-25,no_license,6591249.5,548749.5,monday,street,misdemeanour,M
1,car,EST,RENAULT,2006,65-,dui,6603249.5,545249.5,monday,street,felony,M
2,car,EST,ŠKODA,2017,55-64,speeding,6472249.5,660249.5,monday,street,misdemeanour,M
3,car,EST,VOLKSWAGEN,2004,26-34,dui,6443749.5,567749.5,monday,street,felony,M
4,car,EST,TOYOTA,2012,45-54,traffic_violation,6587749.5,537249.5,monday,street,misdemeanour,M


# Balancing the dataset

In [4]:
data_original['infraction_seriousness'].value_counts()

misdemeanour    367537
felony           14784
Name: infraction_seriousness, dtype: int64

there's a lot of speeding ones which can unbalance the dataset. let's try to balance the dataset by keeping only 70k speeding violations

In [3]:
data_original.head()

misdemeanour_instances = data_original[data_original['infraction_seriousness'] == 'misdemeanour']
felony_instances = data_original[data_original['infraction_seriousness'] == 'felony']


balanced_misdemeanour_instances = misdemeanour_instances.sample(14784, random_state=0)

data_balanced = pd.concat([balanced_misdemeanour_instances, felony_instances])

data_balanced['infraction_seriousness'].value_counts()

felony          14784
misdemeanour    14784
Name: infraction_seriousness, dtype: int64

let's create the balanced dataset

In [4]:
data_x_balanced = data_balanced[['vehicle_brand', 'vehicle_year', 'vehicle_type', 'age_group', 'gender']]
data_x_balanced['vehicle_brand'] = pd.Categorical(data_x_balanced['vehicle_brand']).codes
data_x_balanced['vehicle_year'] = pd.to_numeric(data_x_balanced['vehicle_year'])
data_x_balanced['vehicle_type'] = pd.Categorical(data_x_balanced['vehicle_type']).codes
data_x_balanced['age_group'] = pd.Categorical(data_x_balanced['age_group']).codes
data_x_balanced['gender'] = pd.Categorical(data_x_balanced['gender']).codes


data_y_balanced = data_balanced['infraction_seriousness'].map({'misdemeanour':0, 'felony':1})

/home/sander/Software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sander/Software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sander/Software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

# Model with balanced dataset

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_x_balanced, data_y_balanced, random_state=0)

X_train.shape

(22176, 5)

In [6]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
params = {
    'max_depth': 10,
    'objective': 'reg:logistic',
}
bst = xgb.train(params, dtrain)
y = bst.predict(dtest)

/home/sander/Software/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [7]:
from sklearn.metrics import accuracy_score, classification_report

def label(y):
    if (y >= 0.5):
        return 1
    else:
        return 0
    

y_predicted_labels = list(map(label, y))


print(classification_report(y_test, y_predicted_labels))
print(accuracy_score(y_test, y_predicted_labels))


              precision    recall  f1-score   support

           0       0.69      0.64      0.66      3660
           1       0.67      0.71      0.69      3732

    accuracy                           0.68      7392
   macro avg       0.68      0.68      0.68      7392
weighted avg       0.68      0.68      0.68      7392

0.6768127705627706
